In [21]:
from sentence_transformers import SentenceTransformer, util


class ST:
    def __init__(self, qa_pairs_file: str):
        # Read the file with the QA pairs
        with open(qa_pairs_file, "r", encoding="utf-8") as fIn:
            self.qa_pairs = self.create_sets(fIn.read())

        print("[~] Loading model...")
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.corpus_embeddings = util.normalize_embeddings(
            self.embedder.encode(self.qa_pairs, convert_to_tensor=True).to("cuda")
        )
        print("[+] Model successfully initialised")

    def create_sets(self, content: str):
        lines = content.strip().split("\n")
        qa_pairs: list[tuple[str, str]] = []
        for line in lines:
            if "-" in line:
                qa = line.split("-")
                question = qa[0].strip()
                answer = qa[1].strip()

                # if the answer is just #, use the previous answer
                if answer.startswith("#"):
                    answer = qa_pairs[-1][1]

                qa_pairs.append((question, answer))
        return qa_pairs

    def search(self, q: str) -> str | None:
        # Encode the question
        q_embed = self.embedder.encode([q], convert_to_tensor=True)
        q_embed = q_embed.to("cuda")

        # Find possible hits
        hit = util.semantic_search(
            q_embed, self.corpus_embeddings, score_function=util.dot_score, top_k=1
        )[0][0]

        if hit["score"] < 0.65:
            return None

        return self.qa_pairs[hit["corpus_id"]]


In [30]:
test = ST(qa_pairs_file="/home/mester/Bedlessbot-API/qa_pairs.txt")

[~] Loading model...
[+] Model successfully initialised


In [32]:
test.search("can i ask something")

('Can I ask something?',
 'If you have a question or need help with something, just say it. Asking if you can ask a question is annoying, read [this](https://dontasktoask.com)')

In [27]:
test.search("Tell me about Mester")

('Who is Mester?',
 "just your usual nerd 🤓 but fr, he's an epic guy, you should totally give him a high five or smth")

In [29]:
test.search("how to level up")

('How do I level up?',
 'You level up by talking in text or voice channels. The more active you are, the faster you can progress. There are also random quick time events and periodic XP multiplier events')